In [1]:
UNET = 2

In [2]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH


In [3]:
from textwrap import wrap
import os
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
datasetPathDatabaseAdditional =  configParser.get('COMMON', 'datasetPathDatabase') + '/datasetAdditional.db'
datasetPathDatabaseVgg =  configParser.get('COMMON', 'datasetPathDatabase') + '/datasetFacesBlurred.db'
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
begin_with_image_size = int(configParser.get('COMMON', 'begin_with_image_size'))
unet_dim =  int(configParser.get('COMMON', 'unet_dim'))
unet1_image_size =  int(configParser.get('COMMON', 'unet1_image_size'))
audio_embs =  str(configParser.get('COMMON', 'audio_embs'))
audio_length_used =  configParser.get('train_imagen', 'audio_length_used') 
model_filename =  configParser.get('train_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
sub_epochs=  int(configParser.get('train_imagen', 'sub_epochs') )
batch_size=  int(configParser.get('train_imagen', 'batch_size') )
timesteps= int(configParser.get('COMMON', 'timesteps') )
sample_every=  int(configParser.get('train_imagen', 'sample_every') ) - 1
sample_probability = int(configParser.get('train_imagen', 'sample_probability') )
save_model_every=  int(configParser.get('train_imagen', 'save_model_every') ) - 1
sample_every_offset=  int(configParser.get('train_imagen', 'sample_every_offset') ) - 1
save_every_offset=  int(configParser.get('train_imagen', 'save_every_offset') ) - 1
imagen_samples_folder = configParser.get('train_imagen', 'imagen_samples_folder') 
db_chunk = int(configParser.get('train_imagen', 'db_chunk'))
stop_at_no_of_samples = int(configParser.get('train_imagen', 'stop_at_no_of_samples'))

ignore_speaker_embedding = bool(int(configParser.get('train_imagen', 'ignore_speaker_embedding') ))
ignore_speech_brain = bool(int(configParser.get('train_imagen', 'ignore_speech_brain') ))
ignore_pyannote_titanet_speakernet = bool(int(configParser.get('train_imagen', 'ignore_pyannote_titanet_speakernet') ))
ignore_audio_features = bool(int(configParser.get('train_imagen', 'ignore_audio_features') ))
ignore_pyAudioAnalysis = bool(int(configParser.get('train_imagen', 'ignore_pyAudioAnalysis') ))
ignore_librosa = bool(int(configParser.get('train_imagen', 'ignore_librosa') ))
ignore_image_guide = bool(int(configParser.get('train_imagen', 'ignore_image_guide') ))
ignore_additional_attributes = bool(int(configParser.get('train_imagen', 'ignore_additional_attributes') ))
ignore_age = bool(int(configParser.get('train_imagen', 'ignore_age') ))
ignore_gender = bool(int(configParser.get('train_imagen', 'ignore_gender') ))
ignore_ethnicity = bool(int(configParser.get('train_imagen', 'ignore_ethnicity') ))
ignore_language_spoken = bool(int(configParser.get('train_imagen', 'ignore_language_spoken') ))


pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None



In [4]:
def speaker_emb_preprocess(speaker_emb2):

    if(ignore_speaker_embedding or ignore_speech_brain):
        #print('ignore speech brain')
        return np.zeros((1,768)).tolist()
    else:
        speaker_emb2 = pickle.loads(speaker_emb2)
        #print(speaker_emb2.print)
        speaker_emb2 = speaker_emb2.squeeze()
        speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
        speaker_emb2 = np.tile(speaker_emb2, (1, 1))
        speaker_emb2 = speaker_emb2 / 200.0
        #print(speaker_emb2.shape)
        speaker_emb2 = np.array(speaker_emb2).tolist()
        return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    
    if(ignore_pyannote_titanet_speakernet or ignore_speaker_embedding):
        #print('ignore speech brain')
        return np.zeros((28,768)).tolist()
    else:
        speaker_emb2 = pickle.loads(speaker_emb2)
        a = speaker_emb2 #np.zeros(shape=(24, 512))
        a = a[-28:,:] # Actually only last 39 are the relevant ones
        #print('au')
        #print(a.shape)
        b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
        #c = np.zeros(shape=(28-a.shape[0], 768))
        arr = np.concatenate((a, b), axis=1)
        #arr = np.concatenate((arr, c), axis=0)
        arr = arr / (1 if(audio_embs == 'wav2vec') else 10 if(audio_embs == 'openl3')  else 1 if(audio_embs == 'pyannoteTitaNet') else 1)
        #print(str(arr.max()))
        #print(arr.shape)
        speaker_emb2 = np.array(arr).tolist()
        return speaker_emb2

def audio_features_preprocess(video_id):
    #  79 belong to pyaudioanalysis
    # 111 belogn to liborsa
    if(ignore_audio_features):
        #print('ignore speech brain')
        return np.zeros((190,768)).tolist()
    else:
        #print(video_id)
        conAdditional = sl.connect(datasetPathDatabaseAdditional)
        cur = conAdditional.cursor()
        sql = '''SELECT AUDIO_FEATURES FROM AUDIO WHERE VIDEO_ID = ?'''
        cur.execute(sql, [video_id])
        audio_features = cur.fetchall()
        #print(audio_features[0])
        audio_features = pickle.loads(audio_features[0][0])
        #print(audio_features.shape) # 190 x 128
        audio_features = audio_features[0:190]
        
        #import sys
        #np.set_printoptions(threshold=sys.maxsize)
        #print(np.argwhere(audio_features == 0))

        if(ignore_pyAudioAnalysis and not ignore_librosa):
            zpa = np.zeros((190-110,128))
            audio_features = audio_features[80:190]
            audio_features = np.vstack((audio_features,zpa))
            #print('ignore pyAudioAnalysis')
            #print(audio_features.shape)
        elif(ignore_librosa and not ignore_pyAudioAnalysis):
            zpa = np.zeros((190-80,128))
            audio_features = audio_features[0:80]
            audio_features = np.vstack((audio_features,zpa))
            #print('ignore librosa')
            #print(audio_features.shape)
        else:
            audio_features = np.zeros((190,128))
            #print('ignore pyaudioanalysis and librosa')


        z1 = np.zeros((190,768-128))
        audio_features = np.hstack((audio_features,z1))
        audio_features = audio_features / 100.0
        #print(audio_features.shape)
    return audio_features

def audio_transformer_features_preprocess(video_id):
    #print(video_id)
    conAdditional = sl.connect(datasetPathDatabaseAdditional)
    cur = conAdditional.cursor()
    sql = '''SELECT AUDIO_FEATURES FROM AUDIO_TRANSFORMER WHERE VIDEO_ID = ?'''
    cur.execute(sql, [video_id])
    audio_features = cur.fetchall()
    #print(audio_features[0])
    audio_features = pickle.loads(audio_features[0][0])
    #print(audio_features.shape) # 514 x 768
    audio_features = audio_features.squeeze()
    audio_features = audio_features[0::3]#172*768
    #z1 = np.zeros((161,768-128))
    #audio_features = np.hstack((audio_features,z1))
    #audio_features = audio_features / 100.0
    return audio_features

import random
def image_guide_preprocess(face_id):
    #print(face_id)

    if(random.random() > 2):
        image_guide = np.zeros((49,768))
        #print(image_guide.shape)
    else:
        conAdditional = sl.connect(datasetPathDatabaseVgg)
        cur = conAdditional.cursor()
        sql = '''SELECT BLURRED_FACE_EMB FROM FACES_BLURRED WHERE FACE_ID = ?'''
        cur.execute(sql, [face_id])
        image_guide = cur.fetchall()
        #print(image_guide)
        image_guide = pickle.loads(image_guide[0][0])
        image_guide = image_guide.squeeze() #197 x 768
        image_guide = image_guide[1::4] #49 x 768
    return image_guide


#boxBlurMin =  int(configParser.get('extractVggBlurred', 'boxBlurMin'))
#boxBlurMax =  int(configParser.get('extractVggBlurred', 'boxBlurMax'))

#gaussianBlurMin =  int(configParser.get('extractVggBlurred', 'gaussianBlurMin'))
#gaussianBlurMax =  int(configParser.get('extractVggBlurred', 'gaussianBlurMax'))


from PIL import Image,ImageFilter
import random
#import cv2
import numpy as np
#from matplotlib import pyplot as plt
  


def image_guide_preprocess_low_res_dummy(path):
    #print(face_id)

    image_guide = np.zeros((1, 768))
    return image_guide

def image_guide_preprocess_low_res(path):
    #print(face_id)


    if(random.random() > 2): #never
        image_guide = np.zeros((49, 768))
        #print(image_guide.shape)
    else:

        image = Image.open(path).convert('RGB')

        #print(image.size)

        w_s = image_size / (1+2 * 0.4)
        h_s = image_size / (1+2 * 0.4)

        image = image.crop((0.2*w_s, 0.0*h_s, 1.6*w_s, 1.4*h_s))

        image = image.resize((image_size,image_size))

        image = image.resize((begin_with_image_size,begin_with_image_size))
        im = image
        

        #print('saving')
        #image.save('opop.png')

        #print(np.array(image,np.float32).shape)

        pix = np.array(image, np.float32)
        pix = np.moveaxis(pix, -1, 0)

        pix = pix / 255
        image.close()
        im.close()
        return pix.tolist()
    return image_guide




In [5]:



def process_age(age):
    if(random.random() < 0.2 or ignore_age or ignore_additional_attributes):
        x = np.zeros(768)
        x[767] = 1
        return x
    else:
        x = np.ones(768) * (age / 100.0)
        x[767] = 0
        return x

def process_gender(gender):
    if(random.random() < 0.2 or ignore_gender or ignore_additional_attributes):
        return np.zeros(768)
    elif(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)
    
# TODO
def process_ethnicity(eth):
    x = np.zeros(768)
    if(random.random() < 0.2 or ignore_ethnicity or ignore_additional_attributes):
        x = x
    elif(eth == "indian"):
        x[0] = 1
    elif(eth == "asian"):
        x[16]=1
    elif(eth == "latino hispanic"):
        x[2]=1
    elif(eth == "black"):
        x[3]=1
    elif(eth == "middle eastern"):
        x[4]=1
    elif(eth == "white"):
        x[5]=1 
    else:
        x = x
    return x     

def process_language(lan):
    x = np.zeros(768)
    if(random.random() < 0.2 or ignore_language_spoken or ignore_additional_attributes):
        x = x
    elif(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x       

In [6]:

def process_input(age,ethnicity,gender,language,speaker_emb,audio_emb,audio_features,image_guide):
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((age, ethnicity))
    h = np.vstack((h, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))
    h = np.vstack((h,audio_features))
    h = np.vstack((h,image_guide))
    #print('aaaaaaaaaaa')
    #print(h.shape)
    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    return h.tolist()



def process_image_path(path):
    
    image = Image.open(path).convert('RGB')

    w_s = image_size / (1+2 * 0.4)
    h_s = image_size / (1+2 * 0.4)

    #print(image.size)
    image = image.crop((0.2*w_s,0.0*h_s,1.6*w_s,1.4*h_s))
    image = image.resize((image_size,image_size))

    #print('saving')
    #image.save(str(random.random()) + '.png')

    

    #print(np.array(image,np.float32).shape)
    pix = np.array(image,np.float32)
    pix = np.moveaxis(pix, -1, 0)
    
    pix = pix / 255
    image.close()
    return pix.tolist()

In [7]:
import multiprocessing
import train_imagen_batch

procs = []
stop_now = False


while(not stop_now):

    con = sl.connect(datasetPathDatabase)

    print("Getting Data...")
    ## TODO: NEED TO CHECK F.ID CORRESPONDS TO CORRECT FACE_PATH

    data = con.execute("SELECT V.ID,F.ID,V.VIDEO_PATH, V.AGE CAPTION_A, " + 
                        "V.ETHNICITY CAPTION_E, " +
                        "lower(V.GENDER) CAPTION_G, " +
                            "A.SPEAKER_EMB, "+ ("A.WAV_TO_VEC, " if(audio_embs == 'wav2vec') else "A.AUDIO_EMB2, " if(audio_embs == 'openl3')  else "A.PYANNOTE_TITANET, " if(audio_embs == 'pyannoteTitaNet') else ', ') +
                        "A.AUDIO_FEATURES, " +
                        "A.LANG CAPTION_L, "+
                        "F.FACE_PATH "+
                        "FROM VIDEO V "+
                        "INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID " +
                        "INNER JOIN FACE F ON F.ID = (select ID from FACE f2 where f2.video_id = v.ID ORDER By ID limit 1 ) " + 
                        "WHERE AUDIO_LENGTH = " + audio_length_used + ' ' +
                        ("AND V.ID in (select V2.ID from VIDEO v2 WHERE V2.TRAINED IS NULL AND AUDIO_PRE IN (3,4) AND FACES_PRE = 2 ORDER BY ABS(RANDOM()) LIMIT "+ str(db_chunk) + ")" 
                        if(UNET ==1) else "AND V.ID in (select V2.ID from VIDEO v2 WHERE V2.TRAINED = 1 AND AUDIO_PRE IN (3,4) AND FACES_PRE = 2 ORDER BY ABS(RANDOM()) LIMIT "+ str(db_chunk) + ")") )
    dataGotten = data.fetchall()

    
    print("Data Gotten")

    if(len(dataGotten) == 0):
        break

    

    print("Preprocessing Data...")
    df = pd.DataFrame(dataGotten,columns = ['ID','FACE_ID','VIDEO_PATH','caption_a','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','AUDIO_FEATURES','caption_l','image_path'])
    df["image_guide"] = np.nan




    df = df.sample(frac=1).reset_index(drop=True)



    #print(df.head(10))
    data_frame = df[["ID","FACE_ID","image_path","caption_a","caption_e","caption_g","caption_l"]]
    data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
    data_frame['AUDIO_EMB'] = df['AUDIO_EMB']
    data_frame['image_guide'] = df['image_guide']

    
    data_frame['SPEAKER_EMB'] = data_frame['SPEAKER_EMB'].apply(lambda x: speaker_emb_preprocess(x))
    data_frame['AUDIO_EMB'] = data_frame['AUDIO_EMB'].apply(lambda x: audio_emb_preprocess2(x))
    data_frame['AUDIO_FEATURES'] = data_frame['ID'].apply(lambda x: audio_features_preprocess(x))
    data_frame = data_frame.drop(['ID'], axis=1)
    data_frame['image_guide'] = data_frame['image_path'].apply(lambda x: image_guide_preprocess_low_res_dummy(x))
    data_frame = data_frame.drop(['FACE_ID'], axis=1)
    data_frame['caption_a'] = data_frame['caption_a'].apply(lambda x: process_age(x))
    data_frame['caption_g'] = data_frame['caption_g'].apply(lambda x: process_gender(x))
    data_frame['caption_l'] = data_frame['caption_l'].apply(lambda x: process_language(x))
    data_frame['caption_e'] = data_frame['caption_e'].apply(lambda x: process_ethnicity(x))
    data_frame['low_res_image'] = data_frame['image_path'].apply(lambda x: image_guide_preprocess_low_res(x))

    data_frame['INPUT'] = data_frame['SPEAKER_EMB']

    for index, row in data_frame.iterrows():
        x = process_input(data_frame.loc[index,"caption_a"],data_frame.loc[index,"caption_e"]
                            ,data_frame.loc[index,"caption_g"],
                        data_frame.loc[index,"caption_l"],data_frame.loc[index,"SPEAKER_EMB"],
                        data_frame.loc[index,"AUDIO_EMB"],data_frame.loc[index,"AUDIO_FEATURES"],
                        data_frame.loc[index,"image_guide"])
        x = [x]
        #AADFS = AADFS
        data_frame.loc[index,"INPUT"] = x

    data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)


    data_frame['image_path'] = data_frame['image_path'].apply(lambda x: process_image_path(x))

    input = data_frame['INPUT'].to_numpy()
    input = np.array([np.array(xi) for xi in input])
    input[np.isnan(input)] = 0
    input[input > 10] = 10
    input[input < -10] = -10

    
    output = data_frame['image_path'].to_numpy()
    output = np.array([np.array(xi) for xi in output])
    output.squeeze().shape

    input2 = data_frame['low_res_image'].to_numpy()
    input2 = np.array([np.array(xi) for xi in input2])
    input2.squeeze()

    print("Data Preprocessed...")

    
    #print(len(procs))
    #if(len(procs) > 0):
    #    procs[0].join() # Wait for previous process to finish
    #    print("Model trained using a batch of data...")
    #    procs = []
    #print(len(procs))


    print("Training model using another batch of data...")

    if(UNET == 1):
        proc = multiprocessing.Process(target=train_imagen_batch.train_batch_unet1, args=(input,input2,output,model_filename,sub_epochs,batch_size,sample_every,save_model_every,image_size,unet_dim,timesteps,begin_with_image_size,unet1_image_size,imagen_samples_folder,sample_probability,))
    else:
        proc = multiprocessing.Process(target=train_imagen_batch.train_batch_unet2, args=(input,input2,output,model_filename,sub_epochs,batch_size,sample_every,save_model_every,image_size,unet_dim,timesteps,begin_with_image_size,unet1_image_size,imagen_samples_folder,sample_probability,))
    #procs.append(proc)
    proc.start()
    proc.join()

    if(UNET == 1):
        with open('loss_list_1_temp.pickle', 'rb') as handle:
            loss_list = pickle.load(handle)
    else:
        with open('loss_list_2_temp.pickle', 'rb') as handle:
            loss_list = pickle.load(handle)
    
    if(UNET == 1):
        my_file = Path(model_filename + 'loss_total_1.picke')
        if my_file.is_file():
            with open(model_filename + 'loss_total_1.picke', 'rb') as handle:
                loss_total = pickle.load(handle)
        else:
            loss_total = []
    else:
        my_file = Path(model_filename + 'loss_total_2.picke')
        if my_file.is_file():
            with open(model_filename + 'loss_total_2.picke', 'rb') as handle:
                loss_total = pickle.load(handle)
        else:
            loss_total = []
    
    #print(loss_list)
    #print(loss_total)
    loss_total.extend(loss_list)

    if(UNET == 1):
        with open(model_filename +'loss_total_1.picke', 'wb') as handle:
            pickle.dump(loss_total, handle)
    else:
        with open(model_filename +'loss_total_2.picke', 'wb') as handle:
            pickle.dump(loss_total, handle)


    fig = plt.figure()
    plt.plot(loss_total)
    


    if(os.path.isfile(model_filename + 'loss_1_plot.png')):
        os.remove(model_filename + 'loss_1_plot.png')
    if(UNET == 1):
        if(os.path.isfile(model_filename + 'loss_1_plot.png')):
            os.remove(model_filename + 'loss_1_plot.png')
        fig.savefig(model_filename + 'loss_1_plot.png')
    else:
        if(os.path.isfile(model_filename + 'loss_2_plot.png')):
            os.remove(model_filename + 'loss_2_plot.png')
        fig.savefig(model_filename + 'loss_2_plot.png')
    plt.close()
    

    #print("Model trained using this batch of data...")

    #proc = multiprocessing.Process(target=train_imagen_batch.train_batch_unet2, args=(input,output,model_filename,sub_epochs,batch_size,sample_every,save_model_every,image_size,unet1_dim,unet2_dim,))
    #proc.start()two_unets_pyannote_nemo
    #proc.join()





    
    data.close()

    data5 = df['ID'].to_numpy(dtype=int)
    data6 = []
    for x in data5:
        data6.append(int(x))
    print(len(data6))
    #print(data6)
    if(UNET == 1):
        sql5 = "UPDATE VIDEO SET TRAINED = 5 WHERE ID IN ({0})".format(', '.join('?' for _ in data6))
    else:
        sql5 = "UPDATE VIDEO SET TRAINED = 2 WHERE ID IN ({0})".format(', '.join('?' for _ in data6));
    #print(sql5)
    cur5 = con.cursor()
    cur5.execute(sql5, data6)
    con.commit()
    cur5.close()

    if(UNET == 1):
        data10 = con.execute("SELECT COUNT(*) FROM VIDEO v WHERE V.TRAINED = 1")
    else:
        data10 = con.execute("SELECT COUNT(*) FROM VIDEO v WHERE V.TRAINED = 2")
    dataGotten10 = data10.fetchall()


    print('-------------')

    con.close()

    del dataGotten
    del df
    del data_frame
    del input
    del output
    gc.collect()


    #print(dataGotten10[0])
    if(dataGotten10[0][0] > stop_at_no_of_samples):
        print('--------------- FINISHED ------------------')
        stop_now = True

Getting Data...
Data Gotten
Preprocessing Data...
Data Preprocessed...
Training model using another batch of data...
Training Unet No. 1


2023-10-19 13:20:57.985331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 13:20:58.028149: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


training with dataset of 390 samples and validating with randomly splitted 10 samples
loss: 0.7015090007334948
valid loss: 0.7261835336685181
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
[0.7015090007334948]
[]
400
-------------
(238076,)
Getting Data...
Data Gotten
Preprocessing Data...
Data Preprocessed...
Training model using another batch of data...
Training Unet No. 1


2023-10-19 13:23:14.819199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 13:23:14.863998: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


training with dataset of 390 samples and validating with randomly splitted 10 samples
Using model file imagen_features_low_res_asis_8_24s.pt
checkpoint loaded from imagen_features_low_res_asis_8_24s.pt
loss: 0.6755938101559877
valid loss: 0.5620226860046387
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
[0.6755938101559877]
[0.7015090007334948]
400
-------------
(238076,)
Getting Data...
Data Gotten
Preprocessing Data...
Data Preprocessed...
Training model using another batch of data...
Training Unet No. 1


2023-10-19 13:25:35.083885: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 13:25:35.128066: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


training with dataset of 390 samples and validating with randomly splitted 10 samples
Using model file imagen_features_low_res_asis_8_24s.pt
checkpoint loaded from imagen_features_low_res_asis_8_24s.pt
loss: 0.6526224669069052
valid loss: 0.7486819922924042
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
[0.6526224669069052]
[0.7015090007334948, 0.6755938101559877]
400
-------------
(238076,)
Getting Data...
Data Gotten
Preprocessing Data...
Data Preprocessed...
Training model using another batch of data...
Training Unet No. 1


2023-10-19 13:27:54.476156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 13:27:54.520338: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


training with dataset of 390 samples and validating with randomly splitted 10 samples
Using model file imagen_features_low_res_asis_8_24s.pt
checkpoint loaded from imagen_features_low_res_asis_8_24s.pt
loss: 0.6743548214435577
valid loss: 0.7091527283191681
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
checkpoint saved to imagen_features_low_res_asis_8_24s.pt
[0.6743548214435577]
[0.7015090007334948, 0.6755938101559877, 0.6526224669069052]
400
-------------
(238076,)
Getting Data...
